## Get MLIR files

In [57]:
!cp /content/drive/MyDrive/LicheeRV_Nano_Resources/tpu-mlir_v1.9.beta.0-110-gfdb73ce31-20240719.tar.gz tpu-mlir.tar.gz

In [66]:
!mkdir tpu_mlir
!tar -xzf tpu-mlir.tar.gz -C tpu_mlir --strip-components=1
!mv ./tpu_mlir/$(ls ./tpu_mlir) ./tpu_mlir/source

In [ ]:
!wget https://github.com/sophgo/tpu-mlir/releases/download/v1.14/tpu_mlir-1.14-py3-none-any.whl

## Install python3.10 and tpu_mlir

In [ ]:
!sudo add-apt-repository ppa:deadsnakes/ppa -y

In [ ]:
!sudo apt update
!sudo apt install python3.10

In [ ]:
!wget https://bootstrap.pypa.io/get-pip.py
!python3.10 get-pip.py

In [ ]:
!python3.10 -m pip install tpu_mlir-1.14-py3-none-any.whl
!python3.10 -m pip install flatbuffers
!python3.10 -m pip install ultralytics
!python3.10 -m pip install onnxsim

## Yolo export to ONNX

In [ ]:
!wget https://raw.githubusercontent.com/ret7020/LicheeRVNano/refs/heads/master/Projects/Yolov8/export_to_onnx.py

In [ ]:
# COCO weights for yolov8n.pt
!wget https://github.com/ultralytics/assets/releases/download/v0.0.0/yolov8n.pt

In [ ]:
!python3.10 export_to_onnx.py yolov8n.pt

## Convert to MLIR

In [ ]:
# Image for tests
!wget https://raw.githubusercontent.com/ret7020/YoloBenchmarks/refs/heads/master/assets/images/bus.jpg

In [80]:
!mkdir work
%cd work

/content/work


In [110]:
import os

# Set model input image size
os.environ['IMG_SIZE_W'] = "640"
os.environ['IMG_SIZE_H'] = "640"

# Path to dataset for calibration matrix
os.environ['CALIBRATION_DATASET_PATH'] = "/content/tpu_mlir/source/regression/dataset/COCO2017"
os.environ['CALIBRATION_IMAGES_COUNT'] = "100"

In [ ]:
!export PYTHONPATH=/usr/bin/python3.10
!source ../tpu_mlir/source/envsetup.sh
!model_transform.py --model_name yolov8n --model_def ../yolov8n.onnx --input_shapes [[1,3,$IMG_SIZE_W,$IMG_SIZE_H]] --mean 0.0,0.0,0.0 --scale 0.0039216,0.0039216,0.0039216 --keep_aspect_ratio --pixel_format rgb --test_input ../bus.jpg --test_result yolov8n_top_outputs.npz --mlir yolov8n.mlir

In [ ]:
!source ../tpu_mlir/source/envsetup.sh
!run_calibration.py yolov8n.mlir \
--dataset $CALIBRATION_DATASET_PATH \
--input_num $CALIBRATION_IMAGES_COUNT \
-o yolov8n_cali_table

In [ ]:
!mkdir result
!source ../tpu_mlir/source/envsetup.sh
!model_deploy.py \
--mlir yolov8n.mlir \
--quant_input \
--quant_output \
--quantize INT8 \
--calibration_table yolov8n_cali_table \
--processor cv181x \
--test_input yolov8n_in_f32.npz \
--test_reference yolov8n_top_outputs.npz \
--tolerance 0.85,0.45 \
--model ./result/yolov8n_cv181x_int8_sym.cvimodel